In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

from itertools import islice

In [2]:
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from estimator import PyraformerEstimator

SyntaxError: positional argument follows keyword argument (estimator.py, line 333)

In [ ]:
dataset = get_dataset("electricity")

In [32]:
estimator = PyraformerEstimator(
    freq=dataset.metadata.freq,
    prediction_length=dataset.metadata.prediction_length,
    num_feat_static_cat=1,
    cardinality=[321],

    batch_size=1,
    num_batches_per_epoch=100,
    trainer_kwargs=dict(max_epochs=1, accelerator='gpu', gpus=1),
)

In [33]:
predictor = estimator.train(training_data=dataset.train,num_workers=8)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/parsing.py:245: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/parsing.py:245: UserWarning: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


ValidationError: ignored

In [ ]:
# plt.figure(figsize=(20, 15))
# date_formater = mdates.DateFormatter('%b, %d')
# plt.rcParams.update({'font.size': 15})

# for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
#     ax = plt.subplot(3, 3, idx+1)

#     plt.plot(ts[-4 * dataset.metadata.prediction_length:], label="target", )
#     forecast.plot( color='g')
#     plt.xticks(rotation=60)
#     ax.xaxis.set_major_formatter(date_formater)

# plt.gcf().tight_layout()
# plt.legend()
# plt.show()

In [ ]:
# def plot_prob_forecasts(ts_entry, forecast_entry):
#     plot_length = 70
#     prediction_intervals = (50.0, 90.0)
#     legend = ["observations", "median prediction"] + [f"{k}% prediction interval" for k in prediction_intervals][::-1]

#     fig, ax = plt.subplots(1, 1, figsize=(10, 7))
#     ts_entry[-plot_length:].plot(ax=ax)  # plot the time series
#     forecast_entry.plot(prediction_intervals=prediction_intervals, color='g')
#     plt.grid(which="both")
#     plt.legend(legend, loc="best")
#     plt.show()

In [ ]:
# index = 123
# plot_prob_forecasts(tss[index], forecasts[index])